# 用于测试功能的代码

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+cu118
True


In [3]:
i=1
print(f"ok{i}")


ok1


In [4]:
from bs4 import BeautifulSoup
import xlwt
import requests
import re

def ask_url(url):
    head = {    
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47"
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"
    }
    try:
        r = requests.get(url, headers=head, timeout=30)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        return ""

def get_data(base_url):
    data_list = []
    
    # 遍历每一页
    for i in range(0, 1):
        url = base_url + f"p{i + 1}/"
        html = ask_url(url)
        if html == "":
            continue
        soup = BeautifulSoup(html, 'html.parser')
    
        # 遍历每一种疾病
        for item in soup.find_all('div', class_="result_item"):
            data = {}
            if item.div.p.span.string == "疾病":
                # diseaseName
                data['diseaseName'] = item.div.p.a.string
                
                # diseaseAlias
                # data.append(item.div.p.span.string.strip('（）'))
                
                # symptom
                symptoms = []
                p = item.find('p', class_='result_item_content_label')
                for symptom in p.find_all('a'):
                    symptoms.append(symptom.string)
                
                # https://jbk.39.net/zs/
                sub_url = item.div.p.a.attrs["href"]
                sub_html = ask_url(sub_url)
                if sub_html == "":
                    continue
                sub_soup = BeautifulSoup(sub_html, 'html.parser')
                
                information_ul = sub_soup.find('ul', class_="information_ul")
                for detail in information_ul.find_all('li'):
                    if detail.i.string == '别名：':
                        data['diseaseAlias'] = detail.span.string
                    elif detail.i.string == '发病部位：':
                        data['siteOfOnset'] = []
                        for site in detail.span.find_all('a'):
                            data['siteOfOnset'].append(site.string)
                    elif detail.i.string == '传染性：':
                        data['infectivity'] = detail.span.string
                    elif detail.i.string == '多发人群：':
                        data['multiplePopulation'] = detail.span.string
                    elif detail.i.string == '并发症：':
                        data['complication'] = []
                        for complication in detail.span.find_all('a'):
                            data['complication'].append(complication.string)
                    elif detail.i.string == '挂号科室：':
                        data['registrationDepartment'] = []
                        for department in detail.span.find_all('a'):
                            data['registrationDepartment'].append(department.string)
                    elif detail.i.string == '临床检查：':
                        data['clinicalExamination'] = []
                        for examination in detail.span.find_all('a'):
                            data['clinicalExamination'].append(examination.string)
                    elif detail.i.string == '典型症状：':
                        for symptom in detail.span.find_all('a'):
                            symptoms.append(symptom.string)
                        data['commonDrugs'] = symptoms
                
                information_ul1 = sub_soup.find('ul', class_="information_ul information_ul_bottom")
                for detail in information_ul1.find_all('li'):
                    if detail.i.string == '常用药品：':
                        data['commonDrugs'] = []
                        for drug in detail.span.find_all('a'):
                            data['commonDrugs'].append(drug.string)

            data_list.append(data)
            
    return data_list


def save_data(data_list, save_path):
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet("智能诊断数据集", cell_overwrite_ok=True)
    col = ("diseaseName", "diseaseAlias", "siteOfOnset", "infectivity", "multiplePopulation", "earlySymptom", "advancedSymptom", "complication", "registrationDepartment", "clinicalExamination", "commonDrugs")
    length = len(data_list)
    for i in range(0, 11):
        sheet.write(0, i, col[i])
    for i in range(0, length):
        print("\r当前进度：{:.2f}%".format((i + 1) * 100 / length), end="")
        data = data_list[i]
        for j in range(0, 11):
            if col[j] in data:
                sheet.write(i + 1, j, data[col[j]])
    book.save(save_path)
    return ""

if __name__ == "__main__":
    base_url = "https://jbk.39.net/bw/"
    save_path = ".\\智能诊断数据集.xls"
    # html = ask_url(base_url)
    
    data_list = get_data(base_url)
    save_data(data_list, save_path)


当前进度：100.00%

In [1]:
from bs4 import BeautifulSoup
import requests

def ask_url(url):
    head = {    
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47"
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"
    }
    try:
        r = requests.get(url, headers=head, timeout=30)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        return ""
    
    
for i in range(0, 1):
            url = "https://jbk.39.net/bw/" + f"p{i + 1}/"
            html = ask_url(url)
            if html == "":
                continue
            soup = BeautifulSoup(html, 'html.parser')
        
            # 遍历每一种疾病
            for item in soup.find_all('div', class_="result_item"):
                if item.div.p.span.string == "疾病":
                    # 疾病url
                    disease_url = item.div.p.a.attrs["href"]
                    
                    # 疾病简介
                    disease_jianjie_url = disease_url + "jbzs/" 
                    jianjie_html = ask_url(disease_jianjie_url)
                    if jianjie_html == "":
                        continue
                    jianjie_soup = BeautifulSoup(jianjie_html, 'html.parser')
                    name = jianjie_soup.find('div', class_="disease").h1.string
                    # print(name)
                    jianjie = jianjie_soup.find('p', class_="introduction").string
                    # print(jianjie)
                    for disease_basic in jianjie_soup.find_all('ul', class_="disease_basic"):
                        for li in disease_basic.find_all('li'):
                            if li.span.string == "就诊科室：":
                                span = li.find_all('span')[1]
                                category = ""
                                for a in span.find_all('a'):
                                    category += a.string + ","
                                    department_url = a.attrs["href"]
                                    department_html = ask_url(department_url)
                                    if department_html == "":
                                        continue
                                    department_soup = BeautifulSoup(department_html, 'html.parser')
                                category = category[:-1]
                                # print(category)    
                            elif li.span.string == "相关症状：":
                                span = li.find_all('span')[1]
                                symptom = []
                                for a in span.find_all('a'):
                                    symptom_name = a.string
                                    symptom.append(symptom_name)
                                    
                                    
                                # symptom = symptom[:-1]
                                print(symptom) 
                                
                            
                           
                    
                    

['器质性早泄', '男子性功能障碍', '心理性性功能障碍', '遗精滑泄', '详细']
['湿疣', '外阴鳞状上皮疣状增生', '出血性丘疹', '溃疡分泌物恶臭', '乳头状肿物', '详细']
['椎间盘退行性变', '腰脊椎疼痛', '惧站立，喜依托', '胸腰段及腰椎前凸消失', '根性坐骨神经痛', '详细']
['静脉曲张性外痔', '痔出血', '便血鲜红', '排便障碍', '炎性外痔', '详细']
['心理性性功能障碍', '男子性功能障碍', '雄激素过少', '精冷不育', '脉涩或结', '详细']
['乳腺隐痛', '结节', '囊肿', '经期前乳房痛', '腋窝痛', '详细']
['劳动耐力下降', '胸骨后疼痛', '胸闷', '劳累后心悸', '阵发性夜间呼吸困难', '详细']
['转氨酶增高', '肝肿大', '乙肝表面抗原（HBsAg）阳性', '乙肝e抗原（HBeAg）阳性', '肝功能异常', '详细']
['不明原因发热', '免疫力降低', 'HIV感染', '反复感染', '关节疼痛', '详细']
['剧痒', '风团', '皮肤有压痕', '斑丘疹', '丘疹', '详细']
['第1跖趾关节疼痛和肿大', '关节疼痛', '关节肿胀', '关节畸形', '尿酸盐在关节内沉积增多', '详细']
['腰部钝痛', '肾盏结石', '肾区叩击痛', '排出结石', '腰部包块', '详细']
['胃酸过多', '食欲较差', '腹胀嗳腐', '胃胀', '胃气上逆', '详细']
['口腔粘膜弥漫充血', '口舌生疮', '口腔痛', '溃疡', '容易上火', '详细']
['阴道分泌物增多', '排尿时外阴灼热、瘙痒或疼痛', '性交疼痛', '阴部灼热', '白带增多', '详细']


In [6]:

str = "1,2,3,4,5,"
list = str.split(",")
print(list)


['1', '2', '3', '4', '5', '']


In [ ]:
for item,index in enumerate(list):
    print(item,index)

In [2]:
import pymongo
conn = pymongo.MongoClient('localhost', 27017) # 连接数据库
db = conn['medintel'] # 建立数据库
col = db['test'] # 建表，字典形式

data = {
    'name': '张三',
    'age': 18,
}
col.insert_one(data) # 插入数据
print('插入成功')

插入成功
